In [ ]:
# Gerekli kütüphaneleri yükle
import pandas as pd
import numpy as np
from scipy.sparse import load_npz, csr_matrix
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("MODEL 1: Mutual Information + Random Forest")
print("Hazirlayan: Ismail Can Gunay")
print("="*70)


In [ ]:
# Temizlenmiş dataset'i yükle (tek dosya)
df_clean = pd.read_csv("clean_data.csv")

# Sparse matrisi yükle (tüm özellikler)
X_clean_full = load_npz("X_clean_ready.npz")

# Target değişkeni
y_clean = df_clean['Part 1-2'].copy()
y_clean_binary = (y_clean > 0).astype(int)  # 0: Part 1, 1: Part 2

print(f"Clean data boyutu: {X_clean_full.shape}")
print(f"Target dağılımı: {pd.Series(y_clean_binary).value_counts().to_dict()}")

# Clean data'dan train ve test setlerini ayır (stratified split)
print("\nClean data'dan train ve test setleri ayrılıyor (stratified split)...")
train_indices, test_indices = train_test_split(
    df_clean.index,
    test_size=0.2,  # %20 test, %80 train
    stratify=y_clean_binary,
    random_state=42
)

# Sparse matrisi ve target'ı bu indekslere göre böl
X_train_full = X_clean_full[train_indices]
X_test_full = X_clean_full[test_indices]
y_train_binary = y_clean_binary.iloc[train_indices].values
y_test_binary = y_clean_binary.iloc[test_indices].values

# DataFrame'i de aynı indekslere göre böl
df_train = df_clean.loc[train_indices].reset_index(drop=True)
df_test = df_clean.loc[test_indices].reset_index(drop=True)

print(f"Train set boyutu: {X_train_full.shape}")
print(f"Test set boyutu: {X_test_full.shape}")
print(f"Train target dağılımı: {pd.Series(y_train_binary).value_counts().to_dict()}")

# Stratified sampling (veri çok büyük, hız için örnekleme)
print("\nStratified sampling yapılıyor (200,000 örnek)...")
X_sample, _, y_sample, _ = train_test_split(
    X_train_full, 
    y_train_binary,
    train_size=200000,
    stratify=y_train_binary,
    random_state=42
)
print(f"Sampling sonrası: {X_sample.shape}")
print(f"\nVERİ KULLANIMI ÖZETİ:")
print(f"  - Toplam train verisi: {X_train_full.shape[0]:,} örnek")
print(f"  - Sampling (feature selection için): {X_sample.shape[0]:,} örnek")
print(f"  - Model eğitimi: {X_train_full.shape[0]:,} örnek (tüm train verisi)")
print(f"  - Test verisi: {X_test_full.shape[0]:,} örnek")


## AŞAMA 2: MUTUAL INFORMATION FEATURE SELECTION

**Veri Kullanımı:**
- **Feature Selection:** 200,000 örnek üzerinde çalışıyor
- **Seçilen özellik sayısı:** 150
- **Tüm train/test setlerine uygulanıyor:** 848,024 train / 147,207 test


In [ ]:
# Mutual Information: Her özelliğin target ile bilgi paylaşımını ölçer
# Yüksek MI = Özellik target'ı tahmin etmede daha bilgilendirici
print("Mutual Information skorları hesaplanıyor (bu biraz zaman alabilir)...")

# En iyi 150 özelliği seç
k_best = 150
mi_selector = SelectKBest(score_func=mutual_info_classif, k=k_best)
X_selected = mi_selector.fit_transform(X_sample, y_sample)

# Seçilen özelliklerin indekslerini al
selected_indices = mi_selector.get_support(indices=True)

# Tüm train ve test setlerine uygula
X_train_selected = X_train_full[:, selected_indices]
X_test_selected = X_test_full[:, selected_indices]

print(f"Seçilen özellik sayısı: {len(selected_indices)}")
print(f"Seçilen özelliklerin ilk 10 indeksi: {selected_indices[:10]}")

# MI skorlarını göster
mi_scores = mi_selector.scores_
top_10_indices = np.argsort(mi_scores)[-10:][::-1]
print(f"\nEn yüksek MI skorlarına sahip 10 özellik:")
for idx in top_10_indices:
    print(f"  Özellik {idx}: MI = {mi_scores[idx]:.4f}")

print(f"\nVERİ KULLANIMI:")
print(f"  - Feature selection: {X_sample.shape[0]:,} örnek kullanıldı")
print(f"  - Seçilen özellik sayısı: {len(selected_indices)}")
print(f"  - Train set (seçilen özelliklerle): {X_train_selected.shape}")
print(f"  - Test set (seçilen özelliklerle): {X_test_selected.shape}")


## AŞAMA 3: RANDOM FOREST MODEL EĞİTİMİ

**Veri Kullanımı:**
- **Model Eğitimi:** Tüm train verisi (848,024 örnek) - 150 seçilen özellik ile
- **Model:** Random Forest (200 ağaç)


In [ ]:
# Random Forest: Birden fazla karar ağacının birleşimi
rf_model = RandomForestClassifier(
    n_estimators=200,        # 200 ağaç
    max_depth=20,            # Ağaç derinliği (overfitting'i önlemek için)
    min_samples_split=50,    # Split için minimum örnek sayısı
    min_samples_leaf=20,     # Leaf için minimum örnek sayısı
    random_state=42,
    n_jobs=-1,               # Tüm CPU'ları kullan
    class_weight='balanced'  # Dengesiz sınıflar için ağırlıklandırma
)

print("Model eğitimi başlıyor...")
print(f"Eğitim verisi: {X_train_selected.shape[0]:,} örnek, {X_train_selected.shape[1]} özellik")
rf_model.fit(X_train_selected, y_train_binary)
print("Model eğitimi tamamlandı!")

# Feature importance göster
feature_importance = rf_model.feature_importances_
top_10_features = np.argsort(feature_importance)[-10:][::-1]
print(f"\nEn önemli 10 özellik (Random Forest'e göre):")
for i, idx in enumerate(top_10_features):
    feat_idx = selected_indices[idx]
    print(f"  {i+1}. Özellik {feat_idx}: Importance = {feature_importance[idx]:.4f}")

print(f"\nVERİ KULLANIMI:")
print(f"  - Model eğitimi: {X_train_selected.shape[0]:,} örnek kullanıldı")
print(f"  - Özellik sayısı: {X_train_selected.shape[1]}")


## AŞAMA 4: TEST SETİNE TAHMİN

**Veri Kullanımı:**
- **Test Tahminleri:** 147,207 örnek


In [ ]:
# Test setine tahmin
y_pred_proba = rf_model.predict_proba(X_test_selected)[:, 1]  # Part 2 olasılığı
y_pred = rf_model.predict(X_test_selected)

print(f"Tahmin edilen sınıf dağılımı:")
print(f"  Part 1 (0): {(y_pred == 0).sum()}")
print(f"  Part 2 (1): {(y_pred == 1).sum()}")

print(f"\nVERİ KULLANIMI:")
print(f"  - Test tahminleri: {X_test_selected.shape[0]:,} örnek")


## AŞAMA 5: SUBMISSION DOSYASI OLUŞTURMA


In [ ]:
# Test setindeki DR_NO'ları al
submission_df = pd.DataFrame({
    'DR_NO': df_test['DR_NO'].values if 'DR_NO' in df_test.columns else df_test.index.values,
    'Part 1-2': y_pred
})

# Orijinal değerlere geri çevir
submission_df['Part 1-2'] = submission_df['Part 1-2'].map({
    0: -0.834220,
    1: 1.198725
})

# Dosyayı kaydet
submission_file = "ismailcan_model1_submission.csv"
submission_df.to_csv(submission_file, index=False)
print(f"[OK] Submission dosyasi kaydedildi: {submission_file}")
print(f"   Toplam tahmin sayısı: {len(submission_df)}")

# Model ve selector'ı kaydet
joblib.dump(rf_model, "ismailcan_model1_rf.pkl")
joblib.dump(mi_selector, "ismailcan_model1_selector.pkl")
joblib.dump(selected_indices, "ismailcan_model1_selected_indices.pkl")

print("\n" + "="*70)
print("MODEL 1 TAMAMLANDI!")
print("="*70)
print("\nÖZET:")
print(f"  - Feature Selection: Mutual Information (Top {k_best} features)")
print(f"  - Model: Random Forest (200 trees)")
print(f"  - Submission: {submission_file}")
print("\nVERİ KULLANIMI ÖZETİ:")
print(f"  - Toplam train verisi: {X_train_full.shape[0]:,} örnek")
print(f"  - Sampling (feature selection): {X_sample.shape[0]:,} örnek")
print(f"  - Model eğitimi: {X_train_selected.shape[0]:,} örnek")
print(f"  - Test tahminleri: {X_test_selected.shape[0]:,} örnek")
print("="*70)
